In [1]:
#import packages
import datetime as dt 
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import pandas_datareader
from pandas_datareader import data as pdr
import time
from time import sleep
import string
import yfinance as yf
from bs4 import BeautifulSoup
import urllib
import io
import os
import pandasql
from pandasql import sqldf
import re
import gc
import zipfile
from zipfile import ZipFile
from io import StringIO
import sys

from os import listdir
from os.path import isfile, join
import psutil

from collections import Counter
import linecache
import tracemalloc

import reshape


c:\users\daniel\appdata\local\programs\python\python38-32\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
os.getcwd()

'D:\\GitHub\\finance'

In [3]:
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [4]:
#get ticker, company name and CIK numbers 
ticker_url = "https://www.sec.gov/include/ticker.txt"
ticker_request = requests.get(ticker_url).content
ticker_df = pd.read_csv(io.StringIO(ticker_request.decode('utf-8')),sep="\t")
ticker_df.columns = ['ticker', 'cik']
ticker_df["ticker"] = ticker_df["ticker"].str.lower()


#get sp500 tickers 
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = pd.DataFrame(sp500[0]['Symbol'])
sp500.columns = ['ticker']
sp500["ticker"] = sp500["ticker"].str.lower()
#print(sp500)

#get cik of sp 500
sample = pd.merge(ticker_df,sp500,on='ticker')
print(sample )

    ticker      cik
0      aal     6201
1      aap  1158449
2     aapl   320193
3     abbv  1551152
4      abc  1140859
..     ...      ...
495    yum  1041061
496    zbh  1136869
497   zbra   877212
498   zion   109380
499    zts  1555280

[500 rows x 2 columns]


In [5]:
#get tickers in the sp 500 and get easy stats on them
#get avg price by year, and current price of stock 
def get_easy_stats(df):
    
    easy_df = []
    
    start_dt = dt.datetime(2013,1,1)
    end_dt = dt.datetime(2019,12,31)
    
    for index, row in df.iterrows():
        
        try: 
            sleep(1)
            data_ts = pdr.get_data_yahoo(row['ticker'], start_dt, end_dt)
            data_ts['year'] = data_ts.index.year
            #get average price per share 
            agg_df = data_ts.groupby('year', as_index=False)['Adj Close'].mean()
            agg_df['ticker'] = row['ticker']
            agg_df.columns = ['year','avg_price', 'ticker']
            agg_df['year']=  agg_df['year'].astype(str)
    #       print(agg_df)
            current_price = pdr.get_data_yahoo(row['ticker']).last('1D')['Adj Close'][0].astype(int)

            #turn agg_df wide  for price rows
            wide_df = agg_df.pivot(index='ticker', columns='year', values=['avg_price'])

            wide_df.columns = list(map("".join, wide_df.columns))
            wide_df['current_price'] = current_price
            #print(wide_df)

            easy_df.append(wide_df)
            del agg_df
            del wide_df
        
        except Exception as ex:
            print(str(ex))
            print(row['ticker'] + " failed")
    
    easy_df = pd.concat(easy_df)
    easy_df.to_csv("easy_stats_sp500.csv")
    return(easy_df)
#function end


start = time.time()

#run to generate easy stat csv
easy_stat_df = get_easy_stats(sample)
print(easy_stat_df)

end = time.time()
print("finished getting easy stats for sp500")
print(end - start)

No data fetched for symbol blk using YahooDailyReader
blk failed
'Date'
br failed
No data fetched for symbol cb using YahooDailyReader
cb failed
'Date'
cboe failed
No data fetched for symbol cbre using YahooDailyReader
cbre failed
'Date'
otis failed
        avg_price2013  avg_price2014  avg_price2015  avg_price2016  \
ticker                                                               
aal         17.146522      36.755626      42.940213      36.716199   
aap         83.696200     128.910822     161.370373     153.556964   
aapl        59.771992      83.652771     110.696663      98.462125   
abbv        32.361064      42.512583      49.072086      49.757348   
abc         50.829051      67.690750      95.513013      77.419414   
...               ...            ...            ...            ...   
yum         43.882041      47.608164      52.553995      55.586190   
zbh         75.556204      96.441867     104.284561     109.899370   
zbra        46.445833      71.196667      89.87849

In [6]:
#read in the easy stats of sp500
easy_stat_df = pd.read_csv("easy_stats_sp500.csv")
print(easy_stat_df)

    ticker  avg_price2013  avg_price2014  avg_price2015  avg_price2016  \
0      aal      17.146522      36.755626      42.940213      36.716199   
1      aap      83.696200     128.910822     161.370373     153.556964   
2     aapl      59.771992      83.652771     110.696663      98.462125   
3     abbv      32.361064      42.512583      49.072086      49.757348   
4      abc      50.829051      67.690750      95.513013      77.419414   
..     ...            ...            ...            ...            ...   
489    yum      43.882041      47.608164      52.553995      55.586190   
490    zbh      75.556204      96.441867     104.284561     109.899370   
491   zbra      46.445833      71.196667      89.878492      64.032024   
492   zion      24.429700      26.550984      26.060497      26.514911   
493    zts      30.203858      32.603061      44.496182      46.556359   

     avg_price2017  avg_price2018  avg_price2019  current_price  
0        46.100999      41.898407      30.571

In [7]:
#set up script to traverse directory of quarterly submissions
def generate_finance_table_csv():

    #store all finances in a table
    finance_list = []

    #go into directory with data
    file_os = 'D:\\Finance Data'
    
    
    code_os = 'D:\\GitHub\\finance'
    start = time.time()
    
    #go into file directory
    os.chdir(file_os)
    
    for i in os.listdir():
        file_path = i
        #if its a quarterly filing data, parse it
        if 'q1' in file_path or 'q2' in file_path or 'q3' in file_path or 'q4' in file_path:
            try:
                tracemalloc.start()
                print("starting: " + file_path)

                sub = pd.read_csv(file_path + '/' + 'sub.txt', sep = '\t', encoding = "ISO-8859-1", iterator = True, chunksize =100000)
                sub = pd.concat(sub)
                #clean up form column and filter to just 10-K submissions
                sub['form'] = sub['form'].replace(np.nan, '', regex=True)
                sub = sub[sub['form'] == '10-K']
                #filter to the s&p500
                sub = pd.merge(sub,sample, on = ['cik'])

                pre = pd.read_csv(file_path + '/' + 'pre.txt', sep = '\t', encoding = "ISO-8859-1", chunksize =1000000) 
                num = pd.read_csv(file_path + '/' + 'num.txt', sep = '\t', encoding = "ISO-8859-1",  chunksize =100000)

                print("joining num to sub")
                num_sub = pd.DataFrame()              
                #join in numbers to submissions in chunks
                for chunks in num: 
                    #print(chunks)
                    num_sub = pd.concat([num_sub, sub.merge(chunks, on=['adsh'])])

                #no longer need sub
                del sub

                print("joining pre to numsub")
                nsp = pd.DataFrame()
                #add in plabel and stmt info
                for chunks in pre:
                    #print(chunks)
                    nsp = pd.concat([nsp, num_sub.merge(chunks, on=['adsh', 'tag', 'version'])])
                    nsp = nsp[nsp['stmt'].isin(['BS', 'IS', 'CF', 'CI', 'EQ'])]
                # delete files no longer needed
                del num_sub
                del pre
                del num


                #print(nsp.columns)

                nsp = nsp[['name', 'sic', 'fye', 'form', 'period', 
                           'fy', 'fp', 'filed','ticker', 'cik','ddate', 
                           'qtrs', 'uom', 'value', 'adsh','stmt',  'tag', 
                           'version', 'plabel']]

                #add finances to master table
                finance_list.append(nsp)
                del nsp
                print("processed_data: " + file_path)
                gc.collect()
                snapshot = tracemalloc.take_snapshot()
                display_top(snapshot)
            except Exception as ex:
                print("failed: " + file_path)
                print(str(ex))
    
    #switch back to github directory 
    os.chdir(code_os)
    
    
    finance_table = pd.concat(finance_list)
    del finance_list 
    finance_table.to_csv("total_finance_table.csv")
    end = time.time()
    print("finished traversing files in X secs")
    print(end - start)
    #return(finance_table)

In [8]:
# #run function to generate finance table csv
generate_finance_table_csv()

starting: 2014q1
joining num to sub
joining pre to numsub
processed_data: 2014q1
Top 3 lines
#1: core\algorithms.py:1657: 25078.3 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 14015.1 KiB
    data = self._reader.read(nrows)
#3: internals\managers.py:1848: 7388.1 KiB
    stacked = np.empty(shape, dtype=dtype)
168 other: 1845.4 KiB
Total allocated size: 48327.0 KiB
starting: 2014q2
joining num to sub
joining pre to numsub
processed_data: 2014q2
Top 3 lines
#1: core\algorithms.py:1657: 26137.8 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 16286.1 KiB
    data = self._reader.read(nrows)
#3: internals\managers.py:1848: 11092.2 KiB
    stacked = np.empty(shape, dtype=dtype)
224 other: 2397.1 KiB
Total allocated size: 55913.2 KiB
starting: 2014q3
joining num to sub
joining pre to numsub
processed_data: 2014q3
Top 3 lines
#1: core\algorithms.py:1657: 27837.1 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 11634.5 KiB
    

c:\users\daniel\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


joining pre to numsub
processed_data: 2015q1
Top 3 lines
#1: core\algorithms.py:1657: 55402.8 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 15638.9 KiB
    data = self._reader.read(nrows)
#3: internals\managers.py:1848: 6313.8 KiB
    stacked = np.empty(shape, dtype=dtype)
232 other: 4782.5 KiB
Total allocated size: 82137.9 KiB
starting: 2015q2
joining num to sub
joining pre to numsub
processed_data: 2015q2
Top 3 lines
#1: core\algorithms.py:1657: 55495.7 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 10073.6 KiB
    data = self._reader.read(nrows)
#3: indexes\base.py:763: 3963.8 KiB
    taken = self.values.take(indices)
235 other: 3419.6 KiB
Total allocated size: 72952.7 KiB
starting: 2015q3
joining num to sub
joining pre to numsub
processed_data: 2015q3
Top 3 lines
#1: core\algorithms.py:1657: 57183.8 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 9091.4 KiB
    data = self._reader.read(nrows)
#3: indexes\base.p

<ipython-input-8-41b3537bcb97>:2: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_finance_table_csv()


joining num to sub
joining pre to numsub
processed_data: 2020q2
Top 3 lines
#1: core\algorithms.py:1657: 191544.2 KiB
    out = np.empty(out_shape, dtype=dtype)
#2: io\parsers.py:2037: 30056.2 KiB
    data = self._reader.read(nrows)
#3: indexes\base.py:763: 13679.2 KiB
    taken = self.values.take(indices)
287 other: 6751.7 KiB
Total allocated size: 242031.3 KiB
finished traversing files in X secs
632.563866853714


In [9]:
#main function goes through and tries to parse out the historical data we need: VERY LONG 
def parse_data(df):
    gc.collect()
    ############################################################
    #process data a bit 
    
    #lower case plabel 
    df['plabel'] = df['plabel'].str.lower()
    
    #turn things into an integer
    df['qtrs'] = df.qtrs.astype(int)
    df['period'] = df.period.astype(int)
    df['fy'] = df.fy.astype(int)
    df['fye'] = df.fye.astype(int)
    df['ddate'] = df.ddate.astype(int)
    df['ddate_prev'] = df.ddate.astype(int) + 10000
    
    #turn things to strings
    df['period'] = df['period'].astype(str)
    df['fye'] = df['fye'].astype(str)
    df['fy'] = df['fy'].astype(str)
    df['ddate'] = df['ddate'].astype(str)
    df['ddate_prev'] = df['ddate_prev'].astype(str)
    #pad fye with a leading 0 
    df['fye'] = df['fye'].apply(lambda x: x.zfill(4))
    
    #fill nas inplace = true changes the df directly
    df['period'].fillna('', inplace=True)
    df['fy'].fillna('', inplace=True)
    df['fye'].fillna('', inplace=True)
    df['plabel'].fillna('', inplace=True)
    df['ddate'].fillna('', inplace=True)
    df['ddate_prev'].fillna('', inplace=True)
    
    #create a joinable key
    df['id'] = df['fy'].astype(str) + '_' + df['cik'].astype(str)

    #get distinct company and years
    key_df = pd.DataFrame(df.id.unique(), columns = ['id'])
    gc.collect()
    
    #######################################################################################################
    #get annual revenue 
    revenue_rows = df[
        # get yearly cash flow statements
        (df.stmt.str.contains('IS',na=False))
        & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('revenue',na=False) | df.plabel.str.contains('sales',na=False)) 
    ]
    #get max value for each company and fiscal year
    revenue_agg = revenue_rows.groupby(['id'], as_index=False)['value'].max()  
    revenue_agg.columns = ['id', 'revenue']
    #print(revenue_agg)
    del revenue_rows
    
    #get annual cash flow from operations rows 
    cfo_rows = df[
        # get yearly cash flow statements
        (df.stmt.str.contains('CF',na=False))
        & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('cash',na=False)) 
        & (df.plabel.str.contains('operating',na=False)) 
        ### exclusion words #####
        & (~df.plabel.str.contains('discontinue', na=False))
        & (~df.plabel.str.contains('cost', na=False))
        & (~df.plabel.str.contains('lease', na=False))
        & (~df.plabel.str.contains('adjustments', na=False))
        & (~df.plabel.str.contains('non-cash', na=False))
        & (~df.plabel.str.contains('other', na=False))
        & (~df.plabel.str.contains('disposal', na=False))
        & (~df.plabel.str.contains('escrow', na=False))
        & (~df.plabel.str.contains('investing', na=False))
        & (~df.plabel.str.contains('partnership', na=False))
        & (~df.plabel.str.contains('restricted', na=False))
        & (~df.plabel.str.contains('securities', na=False))
    ]
    #get max value for each company and fiscal year
    cfo_agg = cfo_rows.groupby(['id'], as_index=False)['value'].max()  
    cfo_agg.columns = ['id', 'cfo']
    #print(cfo_agg)
    del cfo_rows
    
    #get annual shares outstanding 
    shares_outstanding_rows = df[
        #get yearly income stmt 
        (df.stmt.str.contains('IS',na=False))
       # & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.uom.str.contains('shares',na=False))
        #inclusion words
        & (df.plabel.str.contains('shares',na=False) | df.plabel.str.contains('stock',na=False) )
        ### exclusion words
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    shares_outstanding_agg = shares_outstanding_rows.groupby(['id'], as_index=False)['value'].max()  
    shares_outstanding_agg.columns = ['id', 'shares_outstanding']
    del shares_outstanding_rows
    
    #get current assets 
    current_assets_rows = df[
        #get yearly balance sheet statements
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('current',na=False)) 
        & (df.plabel.str.contains('asset',na=False)) 
    ]
    current_assets_agg = current_assets_rows.groupby(['id'], as_index=False)['value'].max()  
    current_assets_agg.columns = ['id', 'current_assets']
    del current_assets_rows
    
    #get current liabilites
    current_liabilities_rows = df[
        #get yearly balance sheet statements
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('current',na=False)) 
        & (df.plabel.str.contains('liabilities',na=False)) 
    ]
    current_liabilities_agg = current_liabilities_rows.groupby(['id'], as_index=False)['value'].max()  
    current_liabilities_agg.columns = ['id', 'current_liabilities']
    del current_liabilities_rows
    
    #get depreciation and amortization
    dep_amort_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('depreciation',na=False)) 
        & (df.plabel.str.contains('amortization',na=False)) 
    ]
    dep_amort_agg = dep_amort_rows.groupby(['id'], as_index=False)['value'].max()  
    dep_amort_agg.columns = ['id', 'dep_amort']
    del dep_amort_rows
    
    # try to get depreciation only 
    dep_only_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('depreciation',na=False)) 
    ]
    dep_only_agg = dep_only_rows.groupby(['id'], as_index=False)['value'].max()  
    dep_only_agg.columns = ['id', 'dep_only']
    del dep_only_rows
    
    # try to get amort only
    amort_only_rows = df[
        #get yearly balance sheet statements
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('amortization',na=False)) 
    ]
    amort_only_agg = amort_only_rows.groupby(['id'], as_index=False)['value'].max()  
    amort_only_agg.columns = ['id', 'amort_only']
    del amort_only_rows
    
    #get PPE 
    ppe_rows = df[
        #get yearly income statements
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('property',na=False)) 
        & (df.plabel.str.contains('equipment',na=False)) 
    ]
    ppe_agg = ppe_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_agg.columns = ['id', 'ppe']
    del ppe_rows
    
    #get PPE of the year before
    ppe_prev_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate_prev'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('property',na=False)) 
        & (df.plabel.str.contains('equipment',na=False)) 
    ]
    ppe_prev_agg = ppe_prev_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_prev_agg.columns = ['id', 'ppe_prev']
    del ppe_prev_rows
    
    #get operating income 
    operating_income_rows = df[
        #get yearly income stmt 
        (df.stmt.str.contains('IS',na=False))
        & (df.qtrs == 4)
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('operating',na=False))
        & (df.plabel.str.contains('income',na=False) )
        ### exclusion words
    ]
    operating_income_agg = operating_income_rows.groupby(['id'], as_index=False)['value'].max()  
    operating_income_agg.columns = ['id', 'operating_income']
    del operating_income_rows
    
    #get long term debt on balance sheet
    lt_debt_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('debt',na=False)) 
    ]
    lt_debt_agg = lt_debt_rows.groupby(['id'], as_index=False)['value'].max()  
    lt_debt_agg.columns = ['id', 'lt_debt']
    del lt_debt_rows
    
    #get short term debt on balance sheet
    st_debt_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('st_debt',na=False)) 
    ]
    st_debt_agg = st_debt_rows.groupby(['id'], as_index=False)['value'].max()  
    st_debt_agg.columns = ['id', 'st_debt']
    del st_debt_rows
    
    #get short term debt proceeds from cash flow stmt
    st_debt_proceeds_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        & (df.plabel.str.contains('short',na=False))
        & (df.plabel.str.contains('debt',na=False)) 
        & (df.plabel.str.contains('proceed',na=False))
    ]
    st_debt_proceeds_agg = st_debt_proceeds_rows.groupby(['id'], as_index=False)['value'].max()  
    st_debt_proceeds_agg.columns = ['id', 'st_debt_proceeds']
    del st_debt_proceeds_rows
    
    #get long term debt proceeds  from cash flow stmt
    lt_debt_proceeds_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        &
        (
            (
                (df.plabel.str.contains('long',na=False))
                & (df.plabel.str.contains('debt',na=False)) 
                & (df.plabel.str.contains('proceed',na=False))
            ) | 
            (
            
                (df.plabel.str.contains('issuance',na=False)) 
                & (df.plabel.str.contains('senior notes',na=False))
                & (df.plabel.str.contains('proceed',na=False))
            )
        )
    ]
    lt_debt_proceeds_agg = lt_debt_proceeds_rows.groupby(['id'], as_index=False)['value'].max()  
    lt_debt_proceeds_agg.columns = ['id', 'lt_debt_proceeds']
    del lt_debt_proceeds_rows
    
    #get short term debt repayments from cash flow stmt
    st_debt_repayments_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        & (df.plabel.str.contains('short',na=False))
        & (df.plabel.str.contains('debt',na=False)) 
        & (df.plabel.str.contains('repayment',na=False))
    ]
    st_debt_repayments_agg = st_debt_repayments_rows.groupby(['id'], as_index=False)['value'].max()  
    st_debt_repayments_agg.columns = ['id', 'st_debt_repayments']
    del st_debt_repayments_rows
    
    #get long term debt repayments  from cash flow stmt
    lt_debt_repayments_rows = df[
        (df.stmt.str.contains('CF',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
        #inclusion words
        & (df.plabel.str.contains('long',na=False))
        & (df.plabel.str.contains('debt',na=False)) 
        & (df.plabel.str.contains('repayment',na=False))
    ]
    lt_debt_repayments_agg = lt_debt_repayments_rows.groupby(['id'], as_index=False)['value'].max()  
    lt_debt_repayments_agg.columns = ['id', 'lt_debt_repayments']
    del lt_debt_repayments_rows
    
    #try to get capital expenditure from cash flows 
    
    #get cash and cash equivalents
    cash_rows = df[
        (df.stmt.str.contains('BS',na=False))
        & (df['ddate'] == (df['fy'] + df['fye']))
        #inclusion words
        & (df.plabel.str.contains('cash',na=False)) 
        ### exclusion words #####
        & (~df.plabel.str.contains('discontinue', na=False))
        & (~df.plabel.str.contains('cost', na=False))
        & (~df.plabel.str.contains('lease', na=False))
        & (~df.plabel.str.contains('adjustments', na=False))
        & (~df.plabel.str.contains('other', na=False))
        & (~df.plabel.str.contains('disposal', na=False))
        & (~df.plabel.str.contains('escrow', na=False))
        & (~df.plabel.str.contains('investing', na=False))
        & (~df.plabel.str.contains('partnership', na=False))
        & (~df.plabel.str.contains('restricted', na=False))
        & (~df.plabel.str.contains('securities', na=False))
    ]
    cash_agg = cash_rows.groupby(['id'], as_index=False)['value'].max()  
    cash_agg.columns = ['id', 'cash']
    del cash_rows
    
    #######################################################################################################
    #add financials to key_df
    merged_df = key_df.merge(cfo_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(revenue_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(shares_outstanding_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(current_assets_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(current_liabilities_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(dep_amort_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(dep_only_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(amort_only_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_prev_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(operating_income_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(lt_debt_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(st_debt_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(lt_debt_proceeds_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(st_debt_proceeds_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(lt_debt_repayments_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(st_debt_repayments_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(cash_agg, on = ['id'], how = "left")
    
    
    #######################################################################################################
    #delete unneeded things 
    del cfo_agg
    del revenue_agg
    del shares_outstanding_agg
    del current_assets_agg
    del current_liabilities_agg
    del dep_amort_agg
    del dep_only_agg
    del amort_only_agg
    del ppe_agg
    del ppe_prev_agg
    del operating_income_agg
    del lt_debt_agg
    del st_debt_agg
    del lt_debt_proceeds_agg
    del st_debt_proceeds_agg
    del lt_debt_repayments_agg
    del st_debt_repayments_agg
    del cash_agg 
    
    
    #print(merged_df)
    #print(merged_df.columns)
    return(merged_df)
    #print(key_df)

#     return(result)
    #get all submissions 


In [12]:
# process the data in chunks first, return ticker level dataset 
def init_process_data():
    
    processed_list = []
    
    chunksize = 10 ** 5
    for chunk in pd.read_csv("total_finance_table.csv", chunksize=chunksize):  
        #for each chunk, get finance info on a company, yearly level
        processed_list.append(parse_data(chunk))
    #turn list of dataframes into one dataframe
    processed_table = pd.concat(processed_list)
    
    #split id back into year and cik
    processed_table[['year','cik']] = processed_table.id.str.split(pat = "_", expand=True) 
    print("printing pre-aggregated data")
    print(processed_table)
    del processed_list
    #######################################################################################################
    #aggregate up across chunks 
    agg_df = sqldf(
    """
    SELECT 
    *,
    IFNULL(operating_income,0) + IFNULL(dep_amort,0) AS ebitda, 
    IFNULL(cfo,0) - (IFNULL(ppe,0) - IFNULL(ppe_prev,0)) + IFNULL(lt_debt_proceeds,0) + IFNULL(st_debt_proceeds,0) - 
        IFNULL(lt_debt_repayments,0) - IFNULL(st_debt_repayments,0) AS fcfe
    FROM 
    (
        SELECT  
        year,
        CAST(cik AS INT64) as cik,
        MAX(revenue) AS revenue, 
        MAX(cfo) AS cfo, 
        MAX(shares_outstanding) AS shares_outstanding, 
        MAX(current_assets) AS current_assets,
        MAX(current_liabilities) AS current_liabilities,
        MAX(dep_amort) AS dep_amort,
        MAX(dep_only) AS dep_only,
        MAX(amort_only) AS amort_only,
        MAX(ppe) AS ppe,
        MAX(ppe_prev) AS ppe_prev,
        MAX(operating_income) AS operating_income,
        MAX(lt_debt) AS lt_debt,
        MAX(st_debt) AS st_debt,
        MAX(lt_debt_proceeds) AS lt_debt_proceeds,
        MAX(st_debt_proceeds) AS st_debt_proceeds,
        MAX(lt_debt_repayments) AS lt_debt_repayments,
        MAX(st_debt_repayments) AS st_debt_repayments,
        MAX(cash) AS cash
        FROM 
        processed_table 
        GROUP BY     
        year,
        cik
        ORDER BY 2 DESC
    )

    """)
    
    del processed_table 
    #print(agg_df.columns)
    
    #######################################################################################################
    # turn long data wide 
    wide_df = agg_df.pivot(index='cik', columns='year', values=[ 'ebitda', 'fcfe', 
                                                                'revenue','cfo', 'shares_outstanding', 'current_assets',
                                                                'current_liabilities', 'dep_amort', 'dep_only', 'amort_only', 
                                                                'ppe','ppe_prev', 'operating_income', 
                                                                'lt_debt', 'st_debt',
                                                                'lt_debt_proceeds', 'st_debt_proceeds', 
                                                                'lt_debt_repayments', 'st_debt_repayments', 
                                                                'cash'])
    
    del agg_df
    
    ####################################################################################################### 
    #fix columns
    wide_df.columns = list(map("".join, wide_df.columns))
    
    #add in tickers
    wide_df = wide_df.merge(sample, on = ['cik'])
    
    #drop weird columns 
    wide_df = wide_df[wide_df.columns.drop(list(wide_df.filter(regex='1218')))]
    wide_df = wide_df[wide_df.columns.drop(list(wide_df.filter(regex='2020')))]
    
    #read in easy stats csv
    easy_stat_df = pd.read_csv("easy_stats_sp500.csv")
    
    #join in easy stats
    wide_df = wide_df.merge(easy_stat_df, on = 'ticker')
    
    ####################################################################################################### 
    
    # add dataquality columns - 
    #WTK: how many revenue, ebitda, fcfe and shares_outstanding columns are available with 2019 info available
    
    # WTK: calculated estimated cagrs (FCFE growth, Revenue growth) and multiples (fcfe yield, EV/EBITDA)
    
    # forecast earnings, growth, multiple expansion or decrease over 10 years and get estimated IRR of stock
    
    
    
    ####################################################################################################### 
    #write to csv
    print("printing df to write to csv")
    print(wide_df)
    wide_df.to_csv("sp500_financials.csv")
    return(wide_df)
    
start = time.time()

print("starting to process data")
init_df = init_process_data()

end = time.time()
print("finished processing data")
print(end - start)

starting to process data
printing pre-aggregated data
               id           cfo       revenue  shares_outstanding  \
0       2013_4904  4.106000e+09  1.535700e+10        4.870410e+08   
1       2013_4977  1.054700e+10  2.393900e+10        4.674080e+08   
2       2013_5513  1.031500e+09  1.035380e+10                 NaN   
3       2013_6201  9.490000e+08           NaN        2.763260e+08   
4       2013_7084           NaN  8.980400e+10        6.630000e+08   
..            ...           ...           ...                 ...   
414  2019_1688568  1.783000e+09  2.075300e+10                 NaN   
415  2019_1757898  5.395050e+08  2.782170e+09                 NaN   
416    2019_56873  4.164000e+09  1.218520e+11        8.180000e+08   
417   2019_849399  1.495000e+09  2.456000e+09        6.320000e+08   
418  2020_1613103  7.234000e+09  2.891300e+10        1.351100e+09   

     current_assets  current_liabilities     dep_amort      dep_only  \
0      4.310000e+09         3.421600e+10     

In [16]:
for i in init_df.columns: print(i)
#print(init_df)

cik
ebitda2013
ebitda2014
ebitda2015
ebitda2016
ebitda2017
ebitda2018
ebitda2019
fcfe2013
fcfe2014
fcfe2015
fcfe2016
fcfe2017
fcfe2018
fcfe2019
revenue2013
revenue2014
revenue2015
revenue2016
revenue2017
revenue2018
revenue2019
cfo2013
cfo2014
cfo2015
cfo2016
cfo2017
cfo2018
cfo2019
shares_outstanding2013
shares_outstanding2014
shares_outstanding2015
shares_outstanding2016
shares_outstanding2017
shares_outstanding2018
shares_outstanding2019
current_assets2013
current_assets2014
current_assets2015
current_assets2016
current_assets2017
current_assets2018
current_assets2019
current_liabilities2013
current_liabilities2014
current_liabilities2015
current_liabilities2016
current_liabilities2017
current_liabilities2018
current_liabilities2019
dep_amort2013
dep_amort2014
dep_amort2015
dep_amort2016
dep_amort2017
dep_amort2018
dep_amort2019
dep_only2013
dep_only2014
dep_only2015
dep_only2016
dep_only2017
dep_only2018
dep_only2019
amort_only2013
amort_only2014
amort_only2015
amort_only2016
amort

In [ ]:
# to debug this dataset need tag, plabel, qtrs, stmt and hits descending



In [23]:
#assess quality of data here 
pt_good = sqldf("""SELECT DISTINCT ticker FROM init_df 
                    WHERE operating_income2019 > 0
                    AND ppe2019 IS NOT NULL
                    AND ppe_prev2019 IS NOT NULL
                """)
print(pt_good)

    ticker
0      amd
1     swks
2      hes
3      aep
4      aal
..     ...
192    khc
193     ir
194    bkr
195   avgo
196   amcr

[197 rows x 1 columns]


In [ ]:
#get other ticker info from other script (current price, dividend yield, market cap, shares outsanding , sector)

In [ ]:
#get 2018 walmart ticker = wmt, fy = 2018
# analyze missing info 
dq_df = pd.read_csv("total_finance_table.csv")

In [ ]:
dq_df.columns

In [ ]:
ex = dq_df[(dq_df['ticker'] == 'dis')
           #& (dq_df['fy'] == 2015.0)
           
           ]

In [ ]:
for index, row in ex.iterrows():
    print(row)